# Pneumonia Detection - Model Prediction Testing

This notebook demonstrates how to test your trained model on new chest X-ray images to verify its predictions.

## Import Required Libraries

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

print("✅ All libraries imported successfully!")

## Load Your Trained Model

Make sure you have your best trained model file (.h5) in the current directory.

In [ ]:
# List available model files
model_files = [f for f in os.listdir('.') if f.endswith('.h5')]
print("Available model files:")
for i, file in enumerate(model_files):
    print(f"{i+1}. {file}")

# Load the best model (change filename if needed)
if model_files:
    model_path = model_files[0]  # Use first available model
    print(f"\n🔄 Loading model: {model_path}")
    
    try:
        model = load_model(model_path)
        print(f"✅ Model loaded successfully!")
        print(f"📊 Model architecture: {model.name}")
    except Exception as e:
        print(f"❌ Error loading model: {e}")
else:
    print("❌ No model files found! Please train a model first using pneumonia.ipynb")

## Setup Test Data Generator

Let's create a data generator to load test images systematically.

In [ ]:
# Setup test data generator
test_datagen = ImageDataGenerator(rescale=1./255)

# Try to find test directory
test_dirs = ['./DATASET/chest_xray_balanced/test', './DATASET/chest_xray/test']
test_dir = None

for dir_path in test_dirs:
    if os.path.exists(dir_path):
        test_dir = dir_path
        print(f"✅ Found test directory: {test_dir}")
        break

if test_dir:
    # Create test generator
    test_gen = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=1,
        class_mode='binary',
        shuffle=False  # Keep order for proper comparison
    )
    
    print(f"📊 Test generator created with {test_gen.samples} samples")
    print(f"🏷️ Class indices: {test_gen.class_indices}")
else:
    print("❌ Test directory not found! Please extract DATASET.zip first.")

## Test Single Image Prediction

Let's test the model on individual images to see how it performs.

In [ ]:
def predict_single_image(model, image_path, show_image=True):
    """
    Predict pneumonia for a single image and display results
    """
    # Load and preprocess image
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    
    # Make prediction
    prediction_prob = model.predict(img_array, verbose=0)[0][0]
    
    # Determine class and confidence
    if prediction_prob > 0.5:
        predicted_class = 'PNEUMONIA'
        confidence = prediction_prob
    else:
        predicted_class = 'NORMAL'
        confidence = 1 - prediction_prob
    
    # Get actual class from path
    actual_class = 'PNEUMONIA' if 'PNEUMONIA' in image_path.upper() else 'NORMAL'
    is_correct = predicted_class == actual_class
    
    # Display results
    if show_image:
        plt.figure(figsize=(12, 5))
        
        # Show image
        plt.subplot(1, 2, 1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f'Chest X-ray\n{os.path.basename(image_path)}')
        
        # Show prediction results
        plt.subplot(1, 2, 2)
        plt.axis('off')
        
        # Create text summary
        result_text = f"""
PREDICTION RESULTS

🔮 Predicted: {predicted_class}
✅ Actual: {actual_class}
📊 Confidence: {confidence:.1%}
📈 Raw Score: {prediction_prob:.4f}

{'✅ CORRECT' if is_correct else '❌ INCORRECT'}

Model Logic:
• Score > 0.5 = PNEUMONIA
• Score ≤ 0.5 = NORMAL
"""
        
        plt.text(0.1, 0.5, result_text, fontsize=11, 
                verticalalignment='center',
                bbox=dict(boxstyle="round,pad=0.5", 
                         facecolor='lightgreen' if is_correct else 'lightcoral',
                         alpha=0.8))
        
        plt.tight_layout()
        plt.show()
    
    # Print summary
    print(f"\n📋 {os.path.basename(image_path)}")
    print(f"🔮 Predicted: {predicted_class} ({confidence:.1%})")
    print(f"✅ Actual: {actual_class}")
    print(f"{'✅ CORRECT' if is_correct else '❌ INCORRECT'} - Raw score: {prediction_prob:.4f}")
    
    return {
        'image_path': image_path,
        'predicted_class': predicted_class,
        'actual_class': actual_class,
        'confidence': confidence,
        'raw_probability': prediction_prob,
        'is_correct': is_correct
    }

print("✅ Single image prediction function ready!")

## Test Random Sample Images

Let's test the model on random images from the test set to see how well it performs.

In [ ]:
if test_dir and os.path.exists(test_dir):
    # Get all test image paths
    all_test_images = []
    for root, dirs, files in os.walk(test_dir):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                all_test_images.append(os.path.join(root, file))
    
    print(f"📊 Found {len(all_test_images)} test images")
    
    # Test on random sample
    num_samples = 5  # Change this number to test more/fewer images
    sample_images = random.sample(all_test_images, min(num_samples, len(all_test_images)))
    
    print(f"\n🔍 Testing on {len(sample_images)} random images...")
    print("=" * 60)
    
    results = []
    for i, img_path in enumerate(sample_images, 1):
        print(f"\n[{i}/{len(sample_images)}] Testing: {os.path.basename(img_path)}")
        result = predict_single_image(model, img_path, show_image=True)
        results.append(result)
    
    # Summary
    correct_count = sum(1 for r in results if r['is_correct'])
    accuracy = correct_count / len(results)
    
    print(f"\n📊 SAMPLE TEST RESULTS:")
    print(f"Total tested: {len(results)}")
    print(f"Correct: {correct_count}")
    print(f"Incorrect: {len(results) - correct_count}")
    print(f"Sample Accuracy: {accuracy:.1%}")
    
else:
    print("⚠️ Cannot run test - dataset not found!")

## Full Test Set Evaluation

Let's evaluate the model on the entire test set for comprehensive performance metrics.

In [ ]:
if 'test_gen' in locals():
    print("🔄 Evaluating model on full test set...")
    
    # Reset generator
    test_gen.reset()
    
    # Get predictions for all test images
    predictions = model.predict(test_gen, verbose=1)
    predicted_classes = (predictions > 0.5).astype(int).flatten()
    true_classes = test_gen.classes
    
    # Calculate accuracy
    accuracy = np.mean(predicted_classes == true_classes)
    print(f"\n📊 FULL TEST SET RESULTS:")
    print(f"Total images: {len(true_classes)}")
    print(f"Test Accuracy: {accuracy:.2%}")
    
    # Confusion Matrix
    cm = confusion_matrix(true_classes, predicted_classes)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['NORMAL', 'PNEUMONIA'],
                yticklabels=['NORMAL', 'PNEUMONIA'])
    plt.title(f'Confusion Matrix\nTest Accuracy: {accuracy:.1%}')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.tight_layout()
    plt.show()
    
    # Classification Report
    report = classification_report(true_classes, predicted_classes, 
                                 target_names=['NORMAL', 'PNEUMONIA'])
    print(f"\n📈 DETAILED CLASSIFICATION REPORT:")
    print(report)
    
    # Class-wise accuracy
    normal_mask = true_classes == 0
    pneumonia_mask = true_classes == 1
    
    normal_accuracy = np.mean(predicted_classes[normal_mask] == true_classes[normal_mask])
    pneumonia_accuracy = np.mean(predicted_classes[pneumonia_mask] == true_classes[pneumonia_mask])
    
    print(f"\n🎯 CLASS-WISE PERFORMANCE:")
    print(f"NORMAL accuracy: {normal_accuracy:.1%} ({np.sum(normal_mask)} images)")
    print(f"PNEUMONIA accuracy: {pneumonia_accuracy:.1%} ({np.sum(pneumonia_mask)} images)")
    
else:
    print("⚠️ Cannot run full evaluation - test generator not available!")

## Show Prediction Examples

Let's look at some specific examples of correct and incorrect predictions.

In [ ]:
if 'all_test_images' in locals():
    # Find some correct and incorrect predictions
    print("🔍 Finding examples of correct and incorrect predictions...")
    
    correct_normal = []
    correct_pneumonia = []
    incorrect_predictions = []
    
    # Test a larger sample to find examples
    sample_for_examples = random.sample(all_test_images, min(20, len(all_test_images)))
    
    for img_path in sample_for_examples:
        result = predict_single_image(model, img_path, show_image=False)
        
        if result['is_correct']:
            if result['actual_class'] == 'NORMAL':
                correct_normal.append((img_path, result))
            else:
                correct_pneumonia.append((img_path, result))
        else:
            incorrect_predictions.append((img_path, result))
    
    # Show examples
    print(f"\n✅ Found {len(correct_normal)} correct NORMAL predictions")
    print(f"✅ Found {len(correct_pneumonia)} correct PNEUMONIA predictions")
    print(f"❌ Found {len(incorrect_predictions)} incorrect predictions")
    
    # Display one example from each category if available
    if correct_normal:
        print("\n📸 Example: Correctly identified NORMAL case")
        predict_single_image(model, correct_normal[0][0], show_image=True)
    
    if correct_pneumonia:
        print("\n📸 Example: Correctly identified PNEUMONIA case")
        predict_single_image(model, correct_pneumonia[0][0], show_image=True)
    
    if incorrect_predictions:
        print("\n📸 Example: Incorrect prediction (for analysis)")
        predict_single_image(model, incorrect_predictions[0][0], show_image=True)
    
else:
    print("⚠️ Cannot show examples - test images not loaded!")

## Test Your Own Image

You can test the model on your own chest X-ray image by providing the path below.

In [ ]:
# Test your own image
# Replace 'your_image_path.jpg' with the actual path to your image
custom_image_path = 'your_image_path.jpg'  # Change this path

if os.path.exists(custom_image_path):
    print(f"🔍 Testing custom image: {custom_image_path}")
    result = predict_single_image(model, custom_image_path, show_image=True)
else:
    print("💡 To test your own image:")
    print("1. Upload your chest X-ray image to this directory")
    print("2. Change 'custom_image_path' above to your image filename")
    print("3. Run this cell again")
    
    # Show available image files
    image_files = [f for f in os.listdir('.') if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    if image_files:
        print("\nAvailable image files in current directory:")
        for img in image_files:
            print(f"  - {img}")

## Summary

This notebook demonstrates how to:

1. **Load your trained model** and test it on new images
2. **Make predictions** on individual chest X-ray images
3. **Evaluate model performance** on the test dataset
4. **Visualize results** with confusion matrices and examples
5. **Analyze correct and incorrect predictions** to understand model behavior

### Key Points:
- The model outputs a probability score between 0 and 1
- Score > 0.5 = PNEUMONIA, Score ≤ 0.5 = NORMAL
- Higher confidence scores indicate more certain predictions
- Always validate predictions with medical professionals in real applications

### Next Steps:
- Test the model on more diverse datasets
- Analyze failure cases to improve the model
- Consider ensemble methods for better performance
- Implement Grad-CAM for visual explanations